In [ ]:
import random
import wellytics.api as api
import dotenv
import time

from langchain import OpenAI
from wellytics.models import Form, Question, Metric, Response
from wellytics.utils import uuid

dotenv.load_dotenv()


In [ ]:
_map = lambda f, xs: list(map(f, xs))
_find = lambda f, xs: next(filter(f, xs), None)
_filter = lambda f, xs: list(filter(f, xs))


## Boostrapping

In [ ]:
# class Question(BaseModel):
#     id: str
#     createdAt: int
#     updatedAt: int
#     type: str
#     required: bool
#     question: str
#     placeholder: Optional[str]
#     subQuestions: Optional[List[Dict[str, str]]]
#     options: Optional[List[Dict[str, str]]]
#     min: Optional[int]
#     max: Optional[int]

question_template = "What is your favorite color? {i}"

question_n = 10
question_ids = _map(lambda i: f"question-{i}", range(question_n))
question_texts = _map(lambda i: question_template.format(i=i), range(question_n))
question_dicts = [
    {
        "id": id,
        "createdAt": 0,
        "updatedAt": 0,
        "type": "ShortAnswer",
        "required": True,
        "question": question_texts[i],
        "placeholder": None,
        "subQuestions": None,
        "options": None,
        "min": None,
        "max": None,
    }
    for i, id in enumerate(question_ids)
]
questions = _map(lambda d: Question(**d), question_dicts)

In [ ]:
# class Form(BaseModel):
#     id: str
#     createdAt: int
#     updatedAt: int
#     active: bool
#     title: str
#     description: str
#     questions: List[str]  # ref to question

form_n = 3
form_ids = _map(lambda i: f"form-{i}", range(form_n))
form_questions_n = 3
form_dicts = [
    {
        "id": id,
        "createdAt": 0,
        "updatedAt": 0,
        "active": True,
        "title": f"Form {i}",
        "description": f"Form {i} description",
        "questions": random.sample(question_ids, form_questions_n),
    }
    for i, id in enumerate(form_ids)
]
forms = _map(lambda d: Form(**d), form_dicts)


In [ ]:
tracking_n = 3
tracking_ids = [f"tracking-{i}" for i in range(tracking_n)]


In [ ]:
# class Metric(BaseModel):
#     id: str
#     createdAt: int
#     updatedAt: int
#     trackingId: str
#     values: Dict[str, float]

metric_n = 10
metric_values_n = 3
metric_value_ids = [f"metric-value-{i}" for i in range(metric_values_n)]
metric_ids = []
metric_dicts = []

i = 0
for tracking_id in tracking_ids:
    for _ in range(metric_n):
        id = f"metric-{i}"
        metric_ids.append(id)
        metric_dicts.append(
            {
                "id": id,
                "createdAt": 0,
                "updatedAt": 0,
                "trackingId": tracking_id,
                "values": {
                    metric_value_id: random.random()
                    for metric_value_id in metric_value_ids
                },
            }
        )
        i += 1


metrics = _map(lambda d: Metric(**d), metric_dicts)


In [ ]:
# class Response(BaseModel):
#     id: str
#     createdAt: int
#     updatedAt: int
#     formId: str
#     trackingId: str
#     answers: Dict[str, Union[str, Dict[str, str]]]
#     metrics: List[str]  # ref to metric

response_n = 10
response_metrics_n = 2
response_dicts = []

i = 0
for form_id in form_ids:
    for _ in range(response_n):
        id = f"response-{i}"
        tracking_id = random.choice(tracking_ids)
        form = _find(lambda f: f.id == form_id, forms)
        _metric_ids = [
            metric_id
            for i, metric_id in enumerate(metric_ids)
            if metrics[i].trackingId == tracking_id
        ]

        response_dict = {
            "id": id,
            "createdAt": 0,
            "updatedAt": 0,
            "formId": form_id,
            "trackingId": random.choice(tracking_ids),
            "answers": {
                question_id: random.choice(["Yes", "No"])
                for question_id in form.questions
            },
            "metrics": random.sample(_metric_ids, response_metrics_n),
        }

        response_dicts.append(response_dict)
        i += 1


responses = _map(lambda d: Response(**d), response_dicts)


In [ ]:
for question in questions:
    api.create_question(question)

for form in forms:
    api.create_form(form)

for metric in metrics:
    api.create_metric(metric)

for response in responses:
    api.create_response(response)


## Testing

In [ ]:
api.get_questions()
api.get_question("question-0")
api.get_forms()
api.get_form("form-1")
api.get_metrics()
api.get_metric("metric-0")
api.get_responses("form-1")
api.get_response("form-1", "response-10")


In [ ]:
api.get_form_metrics("form-1")
api.get_response_metrics("form-1", "response-10")
api.get_tracking("tracking-0")


In [ ]:
api.get_response_analytics("form-1", "response-10")

In [ ]:
api.get_form_analytics("form-1")


In [ ]:
question = Question(
    id="new-question",
    createdAt=0,
    updatedAt=0,
    type="ShortAnswer",
    required=True,
    question="What is your favorite color?",
    placeholder=None,
    subQuestions=None,
    options=None,
    min=None,
    max=None,
)
api.create_question(question)

api.add_question_to_form("form-0", "new-question")


In [ ]:
metric = Metric(
    id="new-metric",
    createdAt=0,
    updatedAt=0,
    trackingId="tracking-0",
    values={"metric-value-0": 0.5, "metric-value-1": 0.5, "metric-value-2": 0.5},
)
api.create_metric(metric)


api.add_metric_to_response("form-0", "response-0", "new-metric")


In [ ]:
api.patch_question(
    "new-question",
    {
        "question": "Is this a new question?",
    },
)
api.patch_form(
    "form-0",
    {
        "title": "New title",
    }
)
api.patch_metric(
    "new-metric",
    {
        "values": {"metric-value-0": 1., "metric-value-1": 1.0, "metric-value-2": 1.0},
    }
)
api.patch_response(
    "form-0",
    "response-0",
    {
        "updatedAt": 1,
    }
)


In [ ]:
api.delete_form("form-0")
api.delete_response("form-0", "response-0")

In [ ]:
api.set_form_active("form-1", False)

In [ ]:
api.move_question_up("form-1", "question-0")

In [ ]:
api.move_question_down("form-1", "question-0")

## Fake

In [ ]:
llm = OpenAI()


In [ ]:
questions = [
    "How did you feel before the activity? What were you thinking about? What were you feeling?",
    "How did you feel during the activity? What were you thinking about? What were you feeling?",
    "How did you feel after the activity? What were you thinking about? What were you feeling?",
    "What did you learn about yourself during the activity?",
    "What did you learn about others during the activity?",
]

activity = "Brothers on The Rise held a field trip for 15 boys to Dimond park to learn about the fish and biodiversity of the park. A park guide guided them through a park trail explaining the park's history and evolutionary impacts observed in the wildlife."

prompt = """Answer the following question based on the description of the activity as if you were a boy (12-19 years old). You are from an underrepresented community and are struggling a lot with several several socioeconomic aspects (e.g., COVID-19, poverty, marginalization, etc).

Activity description: {activity_description}

Question: {question}

Answer: """

tracking_n = 5


In [ ]:
now = int(time.time())

_questions = []
for question_text in questions:
    question = Question(
        id=uuid(),
        createdAt=now,
        updatedAt=now,
        type="LongAnswer",
        required=True,
        question=question_text,
        placeholder=None,
        subQuestions=None,
        options=None,
        min=None,
        max=None,
    )
    api.create_question(question)
    _questions.append(question)

_form = Form(
    id=uuid(),
    createdAt=now,
    updatedAt=now,
    active=True,
    title="Activity Reflection",
    description="This form is used to reflect on an activity.",
    questions=[question.id for question in _questions],
)
api.create_form(form)


In [ ]:
tracking_ids = [uuid() for _ in range(response_n)]

_responses = []
for i in range(response_n):
    now = int(time.time())
    answers = {}

    for question in _questions:
        answers[question.id] = (
            llm.generate(
                [
                    prompt.format(
                        activity_description=activity,
                        question=question.question,
                    )
                ]
            )
            .generations[0][0]
            .text
        )

    response = Response(
        id=uuid(),
        createdAt=now,
        updatedAt=now,
        formId=_form.id,
        trackingId=tracking_ids[i],
        answers=answers,
        metrics=[],
    )
    api.create_response(response)
    _responses.append(response)


In [ ]:
api.get_form_analytics(form.id)


In [ ]:
form_analytics = api.get_form_analytics(form.id)


In [ ]:
%pip install wordcloud


In [ ]:
_map = lambda f, xs: list(map(f, xs))
_find = lambda f, xs: next(filter(f, xs), None)
_filter = lambda f, xs: list(filter(f, xs))


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

questionId = "659ceb47-dbd9-4a90-982c-2ea6941faf48"
emotions = form_analytics.emotions[questionId]


question = _find(lambda question: question.id == questionId, _questions)

emotion_labels = [emotion.label for emotion in emotions]
y_pos = np.arange(len(emotion_labels))

emotion_scores = [emotion.score for emotion in emotions]
emotion_random = [random.random() for _ in emotion_labels]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

ax1.bar(y_pos, emotion_random)
ax1.set_xticks(y_pos, emotion_labels, rotation="vertical")
ax1.set_title("(a) Before fine-tuning with domain-specific data")

ax2.bar(y_pos, emotion_scores)
ax2.set_xticks(y_pos, emotion_labels, rotation="vertical")
ax2.set_title("(b) After fine-tuning with domain-specific data")

fig.suptitle(question.question)
plt.savefig("emotion.png")
plt.show()